In [20]:
import numpy as np
import sys
sys.path.append(".././core")

import numpy as np

from gf2 import gf2matrix
# from wiedemann import wiedemann

In [21]:
def berlekamp_massey_algorithm(self, sequence):
    """
    An implementation of the Berlekamp Massey Algorithm. Taken from Wikipedia [1]
    [1] - https://en.wikipedia.org/wiki/Berlekamp-Massey_algorithm
    The Berlekamp–Massey algorithm is an algorithm that will find the shortest linear feedback shift register (LFSR)
    for a given binary output sequence. The algorithm will also find the minimal polynomial of a linearly recurrent
    sequence in an arbitrary field. The field requirement means that the Berlekamp–Massey algorithm requires all
    non-zero elements to have a multiplicative inverse.
    :param block_data:
    :return:
    """
    n = len(sequence)
    c = np.zeros(n)
    b = np.zeros(n)
    c[0], b[0] = 1, 1
    l, m, i = 0, -1, 0
    int_data = [int(el) for el in sequence]
    while i < n:
        v = int_data[(i - l):i]
        v = v[::-1]
        cc = c[1:l + 1]
        d = (int_data[i] + np.dot(v, cc)) % 2
        if d == 1:
            temp = copy.copy(c)
            p = np.zeros(n)
            for j in range(0, l):
                if b[j] == 1:
                    p[j + i - m] = 1
            c = (c + p) % 2
            if l <= 0.5 * i:
                l = i + 1 - l
                m = i
                b = temp
        i += 1
    return l

In [22]:
def wiedemann2Wrapper(A):
    """
    Wrapper for the Wiedemann algorithm using Berlekamp-Massey to find the minimal polynomial.
    Repeats until a minimal polynomial is found. Works over GF(2).
    """
    n = A.n_rows
    while True:
        v = gf2matrix.random(n, 1)
        # Generate the sequence by applying A repeatedly to v
        seq = []
        Av = v
        for _ in range(2 * n):
            seq.append(Av)
            Av = A.apply(Av)
        # Convert the sequence to scalars using a random projection
        u = np.random.randint(0, 2, n).tolist()
        scalar_seq = [int(u.apply(vec)[0, 0]) for vec in seq]
        # Use Berlekamp-Massey to find the minimal polynomial degree
        l = berlekamp_massey_algorithm(scalar_seq)
        if l == n:
            return scalar_seq

def wiedemann2(A, v, d):
    """
    Wiedemann algorithm to compute the minimal polynomial of matrix A using vector v.
    Uses Berlekamp-Massey to find the minimal polynomial from the generated sequence.
    Works over GF(2).
    """
    n = A.n_rows
    seq = []
    Av = v
    for _ in range(2 * n):
        seq.append(Av)
        Av = A.apply(Av)
    u = np.random.randint(0, 2, n).tolist()
    scalar_seq = [int(u.apply(vec)[0, 0]) for vec in seq]
    l = berlekamp_massey_algorithm(scalar_seq)
    return l


In [23]:
def test_scalar_wiedemann():
    """Test the scalar Wiedemann implementation with a known matrix and solution"""
    # Create a matrix and a solution
    n = 5
    A_dense = np.array([
        [1, 1, 0, 0, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1],
        [0, 0, 1, 0, 1],
        [1, 0, 0, 0, 1]
    ], dtype=np.int8)
    
    A = gf2matrix.from_dense(A_dense)
    
    # Choose a true solution
    x_true = np.array([1, 0, 1, 0, 1], dtype=np.int8)
    
    # Calculate the right-hand side b = Ax
    b = A.apply(x_true)
    
    print("Matrix A:")
    print(A)
    print("\nTrue solution x_true:")
    print(x_true)
    print("\nRight-hand side b = Ax:")
    print(b)
    
    # Solve using the Scalar Wiedemann Algorithm
    x_solution = wiedemann2Wrapper(A)
    
    # Check the solution
    if x_solution is not None:
        print("\nComputed solution x_solution:")
        print(x_solution)
        
        # Verify: A * x_solution should equal b
        b_verify = A.apply(x_solution)
        print("\nFinal verification: A * x_solution =")
        print(b_verify)
        print("Matches b:", np.array_equal(b, b_verify))
        print("Matches x_true:", np.array_equal(x_true, x_solution))
    else:
        print("\nNo solution was found.")

# Run the test
test_scalar_wiedemann()

# Also demonstrate with a random sparse matrix
def test_random_matrix():
    """Test the scalar Wiedemann implementation with a random matrix"""
    print("\n\nTesting with a random sparse matrix:")
    
    # Create a random sparse matrix
    n = 8
    A = gf2matrix.random(n, density=0.3)
    
    print("Random sparse matrix A:")
    print(A)
    
    x_solution = wiedemann2Wrapper(A)
    if x_solution is not None:
        print("\nComputed solution x_solution:")
        print(x_solution)
        
        # Verify: A * x_solution should equal 0
        b = A.apply(x_solution)
        print("\nRight-hand side Ax=0:")
        print(b)
        print("Matches zero vector:", np.array_equal(b, np.zeros(n, dtype=np.int8)))
    else:
        print("\nNo solution was found.")

# Run the test with a random matrix
test_random_matrix()

Matrix A:
[[1 1 0 0 1]
 [0 0 0 0 0]
 [0 0 0 1 1]
 [0 0 1 0 1]
 [1 0 0 0 1]]

True solution x_true:
[1 0 1 0 1]

Right-hand side b = Ax:
[0 0 1 0 0]


TypeError: object of type 'gf2matrix' has no len()